# Initialize database

In [1]:
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

# Create a CellMorphologyFromID + access its properties

In [ ]:
morphology_id = "575a84d9-c591-4dda-9789-45ab859047b6"

morphology_from_id = obi.CellMorphologyFromID(id_str=morphology_id)

morphology_from_id.entity(db_client=db_client)
morphology_from_id.swc_file_content(db_client=db_client)
morphology_from_id.neurom_morphology(db_client=db_client)

## Create a CellMorphologyFromID object and pass it to the MorphologyMetricsScanConfig for GridScanGenerationTask execution

In [ ]:
# Create a morphology object (this will fetch a CellMorphology object and create attributes for every parameter of CellMorphology)
morphology_from_id = obi.CellMorphologyFromID(id_str=morphology_id)

# Create a MorphologyMetricsExampleForm.initialize object
initialize = obi.MorphologyMetricsScanConfig.Initialize(morphology=[morphology_from_id, morphology_from_id])

# Create a MorphologyMetricsExampleForm object with the initialize object
scan_config = obi.MorphologyMetricsScanConfig(initialize=initialize)

# Create the grid scan object
scan = obi.GridScanGenerationTask(form=scan_config,
                    coordinate_directory_option="ZERO_INDEX",
                    output_root="../../../../obi-output/morphology_test/grid_scan")

# Run the scan
scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(scan, db_client=db_client)


# Create a CircuitFromID + access its properties

In [2]:
circuit_id = "0182b55e-2f38-4e06-bbd0-b11e70449804"

circuit_from_id = obi.CircuitFromID(id_str=circuit_id)
circuit_entity = circuit_from_id.entity(db_client=db_client)

print()
print(circuit_entity.name)
print(circuit_entity.description)
print(f"#Neurons: {circuit_entity.number_neurons}, #Synapses: {circuit_entity.number_synapses}, #Connections: {circuit_entity.number_connections}")

[2025-10-27 10:24:33,207] INFO: HTTP Request: GET https://staging.openbraininstitute.org/api/entitycore/circuit/0182b55e-2f38-4e06-bbd0-b11e70449804 "HTTP/1.1 200 OK"

nbS1-O1-vSub-sCN-HEX0-L2-01
A volumetric subcircuit (vSub) extracted from the nbS1-O1 circuit, centered on a neuron positioned centrally within the spatial volume of layer 2 in subcolumn HEX0. The subcircuit consists of the central neuron and its nine nearest neighbors, capturing the local connectivity within the parent circuit. The suffix “01” denotes the first instance of such a subnetwork.
#Neurons: 10, #Synapses: 51, #Connections: 10


## Create CircuitFromID objects and pass them to the CircuitExtractionScanConfig for GridScanGenerationTask execution

In [ ]:
# Create a circuit object
circuit_from_id1 = obi.CircuitFromID(id_str="0182b55e-2f38-4e06-bbd0-b11e70449804")
circuit_from_id2 = obi.CircuitFromID(id_str="18ae1ca7-1adc-45c4-a475-635f93446c0e")

# Create a CircuitExtractionScanConfig.initialize object
initialize = obi.CircuitExtractionScanConfig.Initialize(circuit=[circuit_from_id1, circuit_from_id2])

# Create a CircuitExtractionScanConfig object with the initialize object
neuron_set=obi.PredefinedNeuronSet(node_set="Excitatory", sample_percentage=[50, 100])
info = obi.Info(campaign_name="EXC-Extraction", campaign_description="Extraction of percentages of EXC neurons")
scan_config = obi.CircuitExtractionScanConfig(initialize=initialize, neuron_set=neuron_set, info=info)

# Create the grid scan object
scan = obi.GridScanGenerationTask(
    form=scan_config,
    coordinate_directory_option="ZERO_INDEX",
    output_root="../../../../obi-output-v2/circuit_extraction_test/grid_scan"
)

# Run the scan
# use_cache = False  # W/o cache, each circuit will be downloaded separatly for every scan coordinate (and deleted afterwards)
use_cache = True  # With cache, each circuit will only be downloaded once (and kept in the root folder of the scan)
scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(scan, db_client=db_client, entity_cache=use_cache)